# LLMChain
LLM chain 은 off-the-shelf chain 으로 off-the-shelf 는 general 일반적인 목적을 가진 chain 을 의미한다.  
langchain 에 아주 많고 아주 유용하다.

하지만 우리가 스스로 무언가를 만들어볼 때 off-the-shelf chain 보다 우리가 직접 커스텀해서 만든 chain 을 활용하기를 선호할거다.

off-the-shelf chain 은 빠르게 시작할 수 있어서 좋긴 하지만 프레임워크를 다루느라 머리 싸매거나  
off-the-shelf chain을 커스텀하기보다 직접 만들고 싶을 때 이전에 배운 것처럼 langchain expression 언어를 활용해서 우리의 것을 만들 수 있다.

그럼 off-the-shelf chain 은 어떻게 쓰냐면 다음과 같다.
```python
from langchain.chains import LLMChain
```
LLMChain 은 아주 일반적인 것이다.

우리가 가진 memory 를 LLM Chain 안으로 넣어보자.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
)

interaction - 사용자와 AI 사이의 대화(질문과 답변)입니다

예:  
- 사용자: "오늘 날씨 어때?" (토큰 4개)  
- AI: "맑고 화창해요." (토큰 5개)  
위 대화의 총 토큰 수는 9개입니다.  

Interaction의 토큰 수가 많아지면 메모리가 커지면서 비용과 처리 시간이 증가합니다.


memory 는 ConversationSummaryBufferMemory 를 사용해 본다.

ConversationSummaryBufferMemory 는 interaction 들을 기본적으로 가지고 있는 conversation class 이다.
 
그리고 interaction 의 토큰 수가(위의 경우) 80개보다 많으면 가장 오래된 interaction 을 요약해준다.

최신 내용을 그대로 유지하고 대화 기록을 요약하기 시작할거다.

그럼 우리가 원하는 memory 를 사용할 수 있다.

그럼 chain 을 만들어 보자.


In [4]:
from langchain.prompts import PromptTemplate

chain = LLMChain(
    llm=llm, memory=memory, prompt=PromptTemplate.from_template("{question}")
)

chain.predict(question="My name is lution")

'Hello Lution, nice to meet you! How can I assist you today?'

여기에 한 가지 사실을 더 추가해 보자.

In [5]:
chain.predict(question="I live in seoul")

"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

이전 물음에 대한 질문을 해보자.

In [6]:
chain.predict(question="What is my name?")

"I'm sorry, I do not have access to personal information such as your name."

기억하지 못한다.

chain 을 debuging 해보자. langchain 은 verbose 라는 좋은 속성을 가지고 있다.  

이 값을 chain 에 전달하면 chain 이 실행했을 때 chain 의 프롬프트 로그들을 확인할 수 있다.

In [7]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,
)

chain.predict(question="My name is lution")



> Entering new LLMChain chain...
Prompt after formatting:
My name is lution

> Finished chain.


'Hello Lution, nice to meet you! How can I assist you today?'

In [8]:
chain.predict(question="I live in seoul")



> Entering new LLMChain chain...
Prompt after formatting:
I live in seoul

> Finished chain.


"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [9]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, I do not have access to personal information such as your name."

```python
chain.predict(question="What is my name?")
```
"what is my name?" 이라고 물으면 프롬프트도 "what is my name?" 이라고 한다.

여기서 보면 대화의 내역이 프롬프트에 계속 추가되진 않는다.

이게 우리가 해야할 일이다.

우리가 원하는 어떤 방식으로라도 프롬프트에게 대화 기록을 추가해 줘야 한다.

그렇지 않으면 우리가 대화 내역을 말해준 적이 없기 떄문에 이전 대화에 대해서는 다 까먹게 된다.

그런데 재밌는 건 memory 는 계속 업데이트 되고 있다는 사실이다.

In [11]:
prev_memory = memory.load_memory_variables({})
print(prev_memory)

{'history': "System: Lution introduces themselves and mentions they live in Seoul. The AI responds enthusiastically, highlighting the vibrant and bustling nature of the city. It asks Lution what they enjoy most about living in Seoul. The human asks the AI for their name, and after revealing it, the AI greets Lution and offers assistance. The AI expresses admiration for Seoul's rich history and culture, mentioning the variety of activities and experiences the city has to offer.\nHuman: What is my name?\nAI: I'm sorry, I do not have access to personal information such as your name."}


위에서 보듯, 우리의 대화를 요약하는 건 이미 잘 동작하고 있다. 자체적으로 업데이트도 하면서.
 
문제는 이것들은 프롬프트에 포함하지 않는다는 것.

프롬프트에 전달되어야 하는 대화 내용이 LLM 에게 전달되지 않고 지금 프롬프트는 우리에게 혹은 LLM 에게 question 만 주고 있다.

메모리에 좀 더 큰 토큰제한을 걸어보고, 프롬프트도 새로 만들어보자.



AI 가 우리의 대화 기록을 기억하여 question 을 완성할 수 있도록.

In [12]:
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

```python
{'history': "System: Lution introduces themselves and mentions they live in Seoul. The AI responds enthusiastically, highlighting the vibrant and bustling nature of the city. It asks Lution what they enjoy most about living in Seoul. The human asks the AI for their name, and after revealing it, the AI greets Lution and offers assistance. The AI expresses admiration for Seoul's rich history and culture, mentioning the variety of activities and experiences the city has to offer.\nHuman: What is my name?\nAI: I'm sorry, I do not have access to personal information such as your name."}
```
보다시피 history 와 memory 가 human 과 LLM 의 인풋 아웃풋을 기반으로 업데이트 되고 있다.

이제 우리는 이 conversation memory class (ConversationSummaryBufferMemory)에게 템플릿 안에 콘텐츠를 넣으라고 얘기를 해줘야 한다.

이건 memory 에 memory_key 를 불러서 chat_history(위에서 설정한)라고 말해주기만 하면 된다.

memory.load_memory_variables({}) 같은 것들은 안해도 된다.

단지 conversation memory class 에게 memory key 라는 속성을 불러주기만 하면 된다.

* memory_key: template 안에 memory 가 history 를 저장하도록 한 곳을 적어준다. <br>즉 memory_key 와 template 의 메모리 저장 변수명은 동일해야 한다.

memory 는 template 안에 대화 기록을 저장하는 공간을 찾을거고, 자동으로 그 메모리의 기록을 여기에 담는다.

In [14]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is lution")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human: My name is lution
    You: 


> Finished chain.


'Hello lution! How can I assist you today?'

In [15]:
chain.predict(question="I live in seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is lution
AI: Hello lution! How can I assist you today?
    Human: I live in seoul
    You: 


> Finished chain.


"That's great to hear! Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?"

"나는 서울에 산다" 를 실행해본다.

그럼 보다시피 프롬프트에 기록이 남겨져 있다.


In [16]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is lution
AI: Hello lution! How can I assist you today?
Human: I live in seoul
AI: That's great to hear! Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
    Human: What is my name?
    You: 


> Finished chain.


'Your name is lution.'

이제 우리는 memory 를 AI 에게 전달했다.

요약하자면, 우리는 그저 메모리를 위한 공간을 우리 프롬프트 템플릿 안에 만들기만 하면 된다.

```python
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human: {question}
    You: 
"""

{chat_history} 여기에 들어가는 변수의 이름은 아무거나 해도 상관없다. 
```

우린 프롬프트 템플릿에 memory 를 위한 공간을 만들고, 우리가 사용하고 있는 memory class 에게 

history 를 넣을 곳을 알려주기만 하면 된다.(memory_key)

여기까지가 우리가 memory 에 추가하고 싶을때, 문자열 형태로 된 프롬프트 템플릿에 대화 기록을 추가하고 싶을 때 하는 일이였다.